## Taller N°1 : Aplicacion de Algoritmos de B´usqueda

* Profesor: Jorge Ivan Padilla Buritica
* Estudiante: Juan Felipe Cardona Arango 
* Maestria en Ciencia de Datos y Analitica
* Curso Fundamentos de Inteligencia Artificial
* 2025-02


## Preparacion del entorno de trabajo

In [1]:
import math
import heapq
import time
from time import time

# Punto 1:Optimizacion de rutas rurales (Logistica)

## Pregunta orientadora
**¿Cómo llego del nodo de inicio al nodo final gastando lo menos posible?**

---

## Algoritmos usados
- **UCS (Búsqueda de Costo Uniforme)**: explora rutas siempre expandiendo el camino más barato encontrado hasta ahora. Garantiza el camino óptimo si los pesos son positivos.
- **A***: es parecido a UCS, pero usa una **heurística** que le da una idea de qué tan cerca está del destino. Esto le permite ser más rápido sin perder la exactitud si la heurística está bien diseñada.

---

## ¿Qué es la heurística?
Una heurística es una **estimación** del costo que falta para llegar al destino. Por ejemplo, si conocemos las coordenadas de cada punto, podemos usar la **distancia en línea recta** (distancia euclidiana) como estimación.

> Si la línea recta siempre es igual o menor que cualquier ruta real, no se sobreestima el costo y A* seguirá encontrando la mejor ruta.

---

## Solución propuesta
1. Representar el mapa como un grafo con nodos, conexiones y pesos.
2. Elegir el nodo inicial y el nodo destino.
3. Seleccionar el algoritmo (UCS o A*) y, si es A*, definir la heurística.
4. Explorar el grafo paso a paso, comparando costos acumulados.
5. Cuando se llega al destino, devolver el camino encontrado y su costo total.

---

## Medición
Para evaluar un algoritmo no basta con que encuentre el camino óptimo; también importa **el esfuerzo que le costó encontrarlo**.

Por eso medimos:
- **Costo total** del camino encontrado.
- **Nodos expandidos** durante la búsqueda.
- **Profundidad** de la solución (número de pasos o aristas).
- **Tiempo de ejecución** (en milisegundos).



## Pseudocodigo

### UCS

Entrada: grafo G, inicio s, objetivo t, función de costo w(u,v) > 0
Salida: camino óptimo s→t (si existe) y su costo g*(t)

1  open ← PriorityQueue()                      // Cola por costo acumulado g(n)
2  open.push(s, key=0)
3  g[s] ← 0                                     // Costo al inicio es 0
4  parent[s] ← None                             // Para reconstruir el camino
5  depth[s] ← 0
6  visited ← ∅                                  // Conjunto de nodos ya extraídos (cerrados)
7  expanded ← 0                                  // Métrica: conteo de expansiones
8  while open ≠ ∅:
9      (n, _) ← open.pop_min()                  // Extraer nodo con menor g(n)
10     if n ∈ visited:                          // Si ya fue cerrado, saltar
11         continue
12     visited ← visited ∪ {n}
13     expanded ← expanded + 1
14     if n == t:                                // ¡Objetivo alcanzado!
15         return Reconstruir(parent, t), g[t], expanded, depth[t]
16     for cada m ∈ neighbors(n):                // Explorar vecinos
17         costo_tentativo ← g[n] + w(n,m)       // Relajar arista (n,m)
18         if m ∉ g or costo_tentativo < g[m]:   // ¿mejor camino a m?
19             g[m] ← costo_tentativo
20             parent[m] ← n
21             depth[m] ← depth[n] + 1
22             open.push_or_decrease(m, key=g[m])
23  return ∅, ∞, expanded, -1  


### A* 

Entrada: grafo G, inicio s, objetivo t, costo w(u,v) > 0, heurística h(n)
Salida: camino óptimo s→t (si h es admisible y consistente) y su costo g*(t)

1  open ← PriorityQueue()                       // Cola por f(n)=g(n)+h(n)
2  open.push(s, key=h(s))
3  g[s] ← 0
4  parent[s] ← None
5  depth[s] ← 0
6  closed ← ∅                                    // Nodos ya expandidos (cerrados)
7  expanded ← 0
8  while open ≠ ∅:
9      (n, _) ← open.pop_min()                   // Extraer el de menor f
10     if n ∈ closed:                            // Evitar re-procesar cerrados
11         continue
12     closed ← closed ∪ {n}
13     expanded ← expanded + 1
14     if n == t:                                 // Objetivo alcanzado
15         return Reconstruir(parent, t), g[t], expanded, depth[t]
16     for cada m ∈ neighbors(n):                 // Explorar vecinos
17         costo_tentativo ← g[n] + w(n,m)        // Relajar (n,m)
18         if m ∉ g or costo_tentativo < g[m]:    // ¿mejor g para m?
19             g[m] ← costo_tentativo
20             parent[m] ← n
21             depth[m] ← depth[n] + 1
22             f_m ← g[m] + h(m)                  // Clave f para la cola
23             open.push_or_decrease(m, key=f_m)
24  return ∅, ∞, expanded, -1

## Codigo

## Implementacion

In [2]:
from __future__ import annotations
from typing import Dict, Tuple, Callable, List, Any
import math
import time
import heapq
import networkx as nx


In [3]:
Position = Tuple[float, float]
Heuristic = Callable[[Any], float]

class PriorityQueue:
    def __init__(self):
        self._pq = []
        self._entry_finder = {}
        self._REMOVED = object()
        self._counter = 0
    def push_or_decrease(self, item, priority: float):
        if item in self._entry_finder:
            # marca entrada anterior como removida
            old = self._entry_finder.pop(item)
            old[-1] = self._REMOVED
        entry = [priority, self._counter, item]
        self._counter += 1
        self._entry_finder[item] = entry
        heapq.heappush(self._pq, entry)
    def pop_min(self):
        while self._pq:
            priority, _, item = heapq.heappop(self._pq)
            if item is not self._REMOVED:
                self._entry_finder.pop(item, None)
                return item, priority
        raise KeyError('pop from an empty priority queue')
    def empty(self):
        return not self._entry_finder

class SearchResult:
    def __init__(self, path: List[Any], cost: float, expanded: int, depth: int, runtime_ms: float):
        self.path = path
        self.cost = cost
        self.expanded = expanded
        self.depth = depth
        self.runtime_ms = runtime_ms
    def as_dict(self):
        return {
            'path': self.path,
            'cost': self.cost,
            'expanded': self.expanded,
            'depth': self.depth,
            'runtime_ms': round(self.runtime_ms, 3)
        }

# -----------------
# Heurísticas
# -----------------

def h_zero(goal: Any) -> Heuristic:
    return lambda n: 0.0

def h_euclid(G: nx.Graph, goal: Any) -> Heuristic:
    xg, yg = G.nodes[goal]['pos']
    def _h(n):
        x, y = G.nodes[n]['pos']
        return math.hypot(x - xg, y - yg)
    return _h

# Escalada controlada (para estudio de sobreestimación)
# h'(n) = alpha * h_euclid(n), con alpha >= 1

def h_scaled(G: nx.Graph, goal: Any, alpha: float = 1.2) -> Heuristic:
    base = h_euclid(G, goal)
    return lambda n: alpha * base(n)

# -----------------
# Reconstrucción de camino
# -----------------

def reconstruct(parent: Dict[Any, Any], goal: Any):
    path = []
    n = goal
    while n is not None:
        path.append(n)
        n = parent.get(n, None)
    path.reverse()
    return path

# -----------------
# UCS
# -----------------

def ucs(G: nx.Graph, start: Any, goal: Any) -> SearchResult:
    t0 = time.perf_counter()
    openq = PriorityQueue()
    openq.push_or_decrease(start, 0.0)
    g = {start: 0.0}
    parent = {start: None}
    expanded = 0
    visited = set()
    depth = {start: 0}
    while not openq.empty():
        n, _ = openq.pop_min()
        if n in visited:
            continue
        visited.add(n)
        expanded += 1
        if n == goal:
            t1 = time.perf_counter()
            path = reconstruct(parent, goal)
            return SearchResult(path, g[goal], expanded, depth[goal], (t1 - t0)*1000)
        for nbr in G.neighbors(n):
            w = G[n][nbr]['weight']
            tentative = g[n] + w
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                depth[nbr] = depth[n] + 1
                openq.push_or_decrease(nbr, g[nbr])
    return SearchResult([], float('inf'), expanded, -1, (time.perf_counter() - t0)*1000)

# -----------------
# A*
# -----------------

def astar(G: nx.Graph, start: Any, goal: Any, h: Heuristic) -> SearchResult:
    t0 = time.perf_counter()
    openq = PriorityQueue()
    openq.push_or_decrease(start, h(start))
    g = {start: 0.0}
    parent = {start: None}
    depth = {start: 0}
    closed = set()
    expanded = 0
    while not openq.empty():
        n, _ = openq.pop_min()
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            t1 = time.perf_counter()
            path = reconstruct(parent, goal)
            return SearchResult(path, g[goal], expanded, depth[goal], (t1 - t0)*1000)
        for nbr in G.neighbors(n):
            w = G[n][nbr]['weight']
            tentative = g[n] + w
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                depth[nbr] = depth[n] + 1
                f = g[nbr] + h(nbr)
                openq.push_or_decrease(nbr, f)
    return SearchResult([], float('inf'), expanded, -1, (time.perf_counter() - t0)*1000)

# -----------------
# Construcción de grafo de ejemplo
# -----------------

def build_sample_graph() -> Tuple[nx.Graph, Any, Any]:
    G = nx.Graph()
    # nodos con posiciones (x,y)
    coords: Dict[str, Position] = {
        'A': (0, 0), 'B': (2, 1), 'C': (4, 0),
        'D': (1, 3), 'E': (3, 3), 'F': (5, 2)
    }
    for n, pos in coords.items():
        G.add_node(n, pos=pos)
    def euclid(u,v):
        x1,y1 = G.nodes[u]['pos']
        x2,y2 = G.nodes[v]['pos']
        return math.hypot(x1-x2, y1-y2)
    # aristas (puedes ajustar para simular cierres/restricciones)
    edges = [
        ('A','B'), ('B','C'), ('A','D'), ('B','D'),
        ('B','E'), ('C','F'), ('D','E'), ('E','F')
    ]
    for u,v in edges:
        G.add_edge(u, v, weight=euclid(u,v))
    return G, 'A', 'F'

# -----------------
# Runner de experimentos (automatización comparativa)
# -----------------

def run_compare(G: nx.Graph, start: Any, goal: Any, heuristic_name: str = 'euclid', alpha: float = 1.2):
    results = []
    # UCS (baseline)
    r_ucs = ucs(G, start, goal)
    results.append(('UCS', 'h=0', r_ucs))
    # A* con h=0 (equivalente a UCS)
    r_h0 = astar(G, start, goal, h_zero(goal))
    results.append(('A*', 'h=0', r_h0))
    # A* euclidiana
    if heuristic_name == 'euclid':
        r_astar = astar(G, start, goal, h_euclid(G, goal))
        results.append(('A*', 'h=euclid', r_astar))
    # A* con sobreestimación controlada (para análisis)
    r_scaled = astar(G, start, goal, h_scaled(G, goal, alpha=alpha))
    results.append(('A*', f'h={alpha}*euclid', r_scaled))
    # Formato salida
    table = []
    for algo, hname, r in results:
        table.append({
            'algo': algo,
            'heuristic': hname,
            **r.as_dict()
        })
    return table


In [4]:
if __name__ == '__main__':
    G, s, t = build_sample_graph()
    table = run_compare(G, s, t, heuristic_name='euclid', alpha=1.3)
    for row in table:
        print(row)

{'algo': 'UCS', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.15}
{'algo': 'A*', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.022}
{'algo': 'A*', 'heuristic': 'h=euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 5, 'depth': 3, 'runtime_ms': 0.019}
{'algo': 'A*', 'heuristic': 'h=1.3*euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 4, 'depth': 3, 'runtime_ms': 0.015}


## Interpretación de resultados y comparación de algoritmos
A continuación se comentan las métricas obtenidas en el ejemplo para cada algoritmo:

## Resultados obtenidos
| Algoritmo | Heurística       | Camino encontrado       | Costo   | Nodos expandidos | Profundidad | Tiempo (ms) |
|-----------|------------------|------------------------|---------|------------------|-------------|-------------|
| UCS       | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.150       |
| A*        | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.022       |
| A*        | h=euclid         | A → B → C → F           | 6.7082  | 5                | 3           | 0.019       |
| A*        | h=1.3×euclid     | A → B → C → F           | 6.7082  | 4                | 3           | 0.015       |

---

**Interpretación:**
- Todas las configuraciones encontraron el mismo camino óptimo con costo ≈ 6.71.
- Usar una heurística informativa (euclidiana) redujo la cantidad de nodos expandidos y el tiempo de ejecución frente a UCS.
- La heurística escalada (1.3×euclid) aceleró aún más la búsqueda, aunque en grafos más complejos podría perder optimalidad.



## Respuestas a Preguntas Orientadoras

**1) ¿Cómo afecta la calidad de la heurística?**
> Una heurística más informativa (euclidiana) reduce el número de nodos expandidos y el tiempo de ejecución sin perder optimalidad. La heurística escalada (1.3×euclid) reduce aún más las expansiones, pero podría sacrificar optimalidad en casos más complejos.

**2) ¿Es pertinente la euclidiana?**
> Sí, porque el peso de las aristas se calculó como distancia euclidiana. Esto garantiza que la heurística sea admisible y consistente.

**3) ¿Flexibilidad ante nuevas rutas o restricciones?**
> El modelo permite añadir o quitar aristas y ajustar pesos fácilmente. Los algoritmos pueden recalcular rutas óptimas sin modificar la lógica interna.

**4) ¿Escalabilidad si el grafo crece 10× en tamaño?**
> UCS y A* seguirán funcionando, pero el tiempo y memoria crecerán. Una heurística de calidad (como la euclidiana) será clave para reducir expansiones. En grafos muy grandes se recomienda optimizar con técnicas como búsqueda bidireccional o preprocesamiento de rutas.

---

# Punto 2: Red de metro (transporte)
Se proporciona un grafo no ponderado de estaciones de metro. Compare las soluciones usando **BFS** (Breadth-First Search) e **IDS** (Iterative Deepening Search) para ir de la estación A a la estación J.

**Cuestiones críticas**:
- ¿Qué pasa si el sistema crece a 100 nodos? ¿Cuál es el impacto en complejidad temporal?
- ¿Puede automatizarse la elección del mejor algoritmo según el tamaño de la red?

Cada ejercicio debe incluir:
- Análisis del problema y del tipo de búsqueda.
- Implementación en Python y explicación del código.
- Medidas de desempeño (tiempo, nodos expandidos, profundidad y costo).
- Evaluación de complejidad temporal y espacial.
- Rutinas automáticas de comparación entre algoritmos.
- Reflexión crítica sobre eficiencia y aplicabilidad.


## Pseudocodigo 

ALGORITMO Anchura(G, inicio, meta)
    frontera ← Cola()                    // FIFO
    frontera.encolar(inicio)
    explorados ← {inicio}
    padre ← diccionario vacío

    MIENTRAS frontera NO esté vacía HACER
        n ← frontera.desencolar()
        SI n = meta ENTONCES
            DEVOLVER reconstruir_camino(padre, n)
        FIN SI

        PARA CADA vecino EN G.adyacentes(n) HACER
            SI vecino NO está EN explorados ENTONCES
                explorados.agregar(vecino)
                frontera.encolar(vecino)
                padre[vecino] ← n
            FIN SI
        FIN PARA
    FIN MIENTRAS

    DEVOLVER fallo
FIN ALGORITMO

In [ ]:
## Codigo 

In [ ]:
# Punto 3

## Pseudocodigo

Entrada: G=(V,E), inicio s, destino t, weight(u,v) en minutos, h(n)
Salida: ruta óptima y tiempo total

1  open ← PriorityQueue()
2  open.push(s, key=h(s))
3  g[s] ← 0
4  parent[s] ← None
5  closed ← ∅
6  expanded ← 0
7  while open ≠ ∅:
8      (n, _) ← open.pop_min()
9      if n ∈ closed: continue
10     closed ← closed ∪ {n}
11     expanded ← expanded + 1
12     if n == t:
13         return Reconstruir(parent, t), g[t], expanded
14     for cada m ∈ vecinos(n):
15         costo_tent ← g[n] + weight(n,m)
16         if m ∉ g or costo_tent < g[m]:
17             g[m] ← costo_tent
18             parent[m] ← n
19             f_m ← g[m] + h(m)
20             open.push_or_decrease(m, f_m)
21 return ∅, ∞, expanded




In [ ]:
## Codigo

In [7]:
import networkx as nx
import math

# Construcción del grafo de la red de metro
def build_metro_graph():
    G = nx.Graph()
    coords = {
        'A': (0, 0),
        'B': (2, 0),
        'C': (4, 0),
        'D': (4, 2)
    }
    for station, pos in coords.items():
        G.add_node(station, pos=pos)
    edges = [
        ('A', 'B', 3),
        ('B', 'C', 4),
        ('C', 'D', 5)
    ]
    for u, v, time in edges:
        G.add_edge(u, v, weight=time)
    return G, 'A', 'D'

# Heurística basada en línea recta y velocidad promedio (40 km/h ≈ 0.667 km/min)
def h_metro(G, goal):
    vel = 0.667
    xg, yg = G.nodes[goal]['pos']
    def h(n):
        x, y = G.nodes[n]['pos']
        dist = math.hypot(x - xg, y - yg)
        return dist / vel
    return h

# Ejecución de A*
from heapq import heappush, heappop

def astar_metro(G, start, goal, h):
    openq = []
    heappush(openq, (h(start), start))
    g = {start: 0}
    parent = {start: None}
    expanded = 0
    closed = set()
    while openq:
        _, n = heappop(openq)
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            return reconstruct(parent, goal), g[goal], expanded
        for nbr in G.neighbors(n):
            time_cost = G[n][nbr]['weight']
            tentative = g[n] + time_cost
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                heappush(openq, (g[nbr] + h(nbr), nbr))
    return [], float('inf'), expanded

def reconstruct(parent, goal):
    path = []
    n = goal
    while n is not None:
        path.append(n)
        n = parent.get(n)
    return path[::-1]

# Ejemplo de ejecución
G, s, t = build_metro_graph()
path, cost, expanded = astar_metro(G, s, t, h_metro(G, t))
print("Ruta:", path)
print("Tiempo total (min):", round(cost, 2))
print("Nodos expandidos:", expanded)

Ruta: ['A', 'B', 'C', 'D']
Tiempo total (min): 12
Nodos expandidos: 4


## Respuestas a Preguntas Orientadoras

# Punto 3 :Filogenia biologia

Dada una red de metro representada como un grafo ponderado (nodos = estaciones, aristas = tramos entre estaciones), encontrar la ruta de menor tiempo entre dos estaciones usando el algoritmo A* con una heurística apropiada.

Se requiere encontrar la ruta de menor tiempo entre dos estaciones de metro en un grafo ponderado, considerando:

Tramos con tiempos variables.

Posibles penalizaciones por transferencia.

Posibles cierres de tramos.

El problema es una instancia del camino mínimo en grafos ponderados con pesos positivos, para lo cual se emplea A* con una heurística admisible que acelera la búsqueda respecto a UCS.


## Pseudocodigo

In [14]:
Entrada: G=(V,E), inicio s, destino t, weight(u,v) en minutos, h(n)
Salida: ruta óptima y tiempo total

1  open ← PriorityQueue()                       // Cola priorizada por f(n) = g(n) + h(n)
2  open.push(s, key = h(s))                      // Comenzar desde el inicio con costo heurístico
3  g[s] ← 0                                      // Costo acumulado desde el inicio
4  parent[s] ← None                              // Predecesor para reconstruir ruta
5  closed ← ∅                                    // Conjunto de nodos ya visitados
6  expanded ← 0                                  // Contador de nodos expandidos
7  while open ≠ ∅:
8      (n, _) ← open.pop_min()                   // Extraer nodo con menor f(n)
9      if n ∈ closed: continue                   // Saltar si ya fue procesado
10     closed ← closed ∪ {n}
11     expanded ← expanded + 1
12     if n == t:                                // Si es el destino
13         return Reconstruir(parent, t), g[t], expanded
14     for cada m ∈ vecinos(n):                  // Explorar vecinos
15         costo_tent ← g[n] + weight(n, m)       // Costo acumulado tentativo
16         if m ∉ g or costo_tent < g[m]:         // Si es mejor que el anterior
17             g[m] ← costo_tent
18             parent[m] ← n
19             f_m ← g[m] + h(m)                  // Prioridad combinando g y h
20             open.push_or_decrease(m, f_m)      // Insertar o actualizar prioridad
21 return ∅, ∞, expanded                         // No se encontró ruta


SyntaxError: invalid character '←' (U+2190) (1015277404.py, line 4)

## Codigo

In [ ]:
from collections import defaultdict, deque
import time

# ---------- Construcción del árbol ----------
# Entrada: lista de aristas (parent, child)
# Devuelve:
# - parents: dict node -> list of parents
# - children: dict node -> list of children
# - root: nodo raíz inferido (sin padres)
def build_tree(edges):
    parents = defaultdict(list)
    children = defaultdict(list)
    nodes = set()
    for p,c in edges:
        parents[c].append(p)
        children[p].append(c)
        nodes.add(p); nodes.add(c)
    roots = [n for n in nodes if len(parents[n]) == 0]
    root = roots[0] if roots else None
    return dict(parents), dict(children), root

# ---------- Profundidades desde la raíz ----------
# Calcula depth por BFS desde la raíz (soporta DAG/árbol, asumiendo aciclicidad)
def compute_depths(children, root):
    depth = defaultdict(lambda: float('inf'))
    depth[root] = 0
    q = deque([root])
    while q:
        u = q.popleft()
        for v in children.get(u, []):
            # tomar la mínima profundidad conocida
            if depth[u] + 1 < depth[v]:
                depth[v] = depth[u] + 1
                q.append(v)
    return dict(depth)

# ---------- Ancestros hasta L (ascendente) ----------
# Retorna set de nodos alcanzables (incluye el propio) y número de nodos explorados
def ancestors_up_to_L(parents, start, L):
    visited = set([start])
    frontier = [start]
    explored = 0
    steps = 0
    while frontier and steps < L:
        new_frontier = []
        for u in frontier:
            for p in parents.get(u, []):
                explored += 1
                if p not in visited:
                    visited.add(p)
                    new_frontier.append(p)
        frontier = new_frontier
        steps += 1
    return visited, explored

# ---------- MRCA por DFS limitado (ascendente) ----------
# Devuelve: (mrca, depth[mrca] o None, nodos_expandidos, tiempo_ms)
def lca_dls_asc(parents, depth, a, b, L):
    t0 = time.perf_counter()
    A, expA = ancestors_up_to_L(parents, a, L)
    B, expB = ancestors_up_to_L(parents, b, L)
    common = A.intersection(B)
    if not common:
        return None, None, expA + expB, (time.perf_counter() - t0)*1000
    # elegir el ancestro común con mayor profundidad
    mrca = max(common, key=lambda n: depth.get(n, -1))
    t1 = time.perf_counter()
    return mrca, depth.get(mrca, None), expA + expB, (t1 - t0)*1000

# ---------- Automatización para múltiples pares ----------
# pairs: lista de tuplas [(a,b), ...]
# Devuelve lista de dicts con resultados

def batch_lca_dls(parents, depth, pairs, L):
    out = []
    for a,b in pairs:
        mrca, d_mrca, explored, t_ms = lca_dls_asc(parents, depth, a, b, L)
        out.append({
            'sp1': a,
            'sp2': b,
            'L': L,
            'MRCA': mrca,
            'depth_MRCA': d_mrca,
            'expanded': explored,
            'runtime_ms': round(t_ms, 3),
            'status': 'ok' if mrca is not None else 'not_found'
        })
    return out

In [ ]:
# Árbol (parent -> child)
edges = [
    ('R', 'A'), ('R', 'B'),
    ('A', 'C'), ('A', 'D'),
    ('B', 'E'), ('B', 'F'),
    ('D', 'G'), ('E', 'H')
]

parents, children, root = build_tree(edges)
depth = compute_depths(children, root)

pairs = [('C','G'), ('C','H'), ('G','F')]
for L in [1, 2, 3, 10]:
    rows = batch_lca_dls(parents, depth, pairs, L)
    print(f"\nResultados para L={L}")
    for r in rows:
        print(r)


Resultados para L=1
{'sp1': 'C', 'sp2': 'G', 'L': 1, 'MRCA': None, 'depth_MRCA': None, 'expanded': 2, 'runtime_ms': 0.383, 'status': 'not_found'}
{'sp1': 'C', 'sp2': 'H', 'L': 1, 'MRCA': None, 'depth_MRCA': None, 'expanded': 2, 'runtime_ms': 0.044, 'status': 'not_found'}
{'sp1': 'G', 'sp2': 'F', 'L': 1, 'MRCA': None, 'depth_MRCA': None, 'expanded': 2, 'runtime_ms': 0.004, 'status': 'not_found'}

Resultados para L=2
{'sp1': 'C', 'sp2': 'G', 'L': 2, 'MRCA': 'A', 'depth_MRCA': 1, 'expanded': 4, 'runtime_ms': 0.011, 'status': 'ok'}
{'sp1': 'C', 'sp2': 'H', 'L': 2, 'MRCA': None, 'depth_MRCA': None, 'expanded': 4, 'runtime_ms': 0.002, 'status': 'not_found'}
{'sp1': 'G', 'sp2': 'F', 'L': 2, 'MRCA': None, 'depth_MRCA': None, 'expanded': 4, 'runtime_ms': 0.002, 'status': 'not_found'}

Resultados para L=3
{'sp1': 'C', 'sp2': 'G', 'L': 3, 'MRCA': 'A', 'depth_MRCA': 1, 'expanded': 5, 'runtime_ms': 0.005, 'status': 'ok'}
{'sp1': 'C', 'sp2': 'H', 'L': 3, 'MRCA': 'R', 'depth_MRCA': 0, 'expanded': 5,

Interpretación rápida

Efecto de L:

Con L=1 no se alcanza el MRCA en ningún par (todos not_found), pues el ancestro común está a más de 1 nivel para al menos una de las especies.

Con L=2, aparece A como MRCA de (C,G) (profundidad 1), pero aún no alcanza para pares que requieren subir hasta la raíz (R).

Con L=3, ya se recuperan los MRCAs faltantes: R (profundidad 0) para (C,H) y (G,F).

Con L=10 no hay cambios respecto a L=3 (mismo MRCA, igual número de nodos expandidos), lo que sugiere que L=3 es suficiente para este árbol.

Costo computacional: los nodos expandidos crecen con L (de 2→4→5) y el tiempo permanece muy bajo en este caso de tamaño pequeño. En árboles mayores, el crecimiento con L será más notable

## Respuestas a Preguntas Orientadoras

¿Cómo afecta la elección de profundidad límite?

Define hasta qué ancestros se explora. Si L es muy bajo, puedes obtener falsos negativos (no hallar MRCA) o quedarte con un ancestro común más alto. Con L suficiente, recuperas el MRCA real.

¿Podría automatizarse este análisis para múltiples pares?

Sí: batch_lca_dls(...) procesa una lista de pares y devuelve una tabla con MRCA, profundidad, nodos expandidos y tiempo.

¿Qué tan escalable sería con cientos de especies?

El método escala linealmente con los ancestros visitados (≈ O(L·b) por pareja, con b ramificación media). Para cientos/miles de especies y muchas consultas, conviene preprocesar LCA (p. ej., Binary Lifting u Euler Tour + RMQ) que dan consultas O(1)/O(log N) tras O(N log N) de preprocesamiento.

# Punto 4:Evaluacion de emergencia ( Infraestructura )

En un grafo ponderado donde el peso representa nivel de riesgo vial, encontrar con Búsqueda de Costo Uniforme (UCS) la ruta más segura (riesgo total mínimo) desde un punto de origen hacia una o varias salidas.



## Pseudocodigo

In [ ]:
Entrada: G=(V,E), origen s, salidas T, risk(u,v) ≥ 0
Salida: camino s→t* (t*∈T) con riesgo mínimo y su costo

1  open ← PriorityQueue()                          // clave = g(n): riesgo acumulado
2  open.push(s, key=0)
3  g[s] ← 0
4  parent[s] ← None
5  visited ← ∅
6  expanded ← 0
7  while open ≠ ∅:
8      (n, _) ← open.pop_min()                     // nodo con menor g(n)
9      if n ∈ visited: continue
10     visited ← visited ∪ {n}
11     expanded ← expanded + 1
12     if n ∈ T:                                    // se alcanzó la primera salida óptima
13         return Reconstruir(parent, n), g[n], expanded
14     for cada m ∈ vecinos(n):
15         costo ← g[n] + risk(n,m)
16         if m ∉ g or costo < g[m]:
17             g[m] ← costo
18             parent[m] ← n
19             open.push_or_decrease(m, key=g[m])
20  return ∅, ∞, expanded                           // no hay ruta disponible


## Codigo

In [15]:
from heapq import heappush, heappop
from collections import defaultdict
import math, time

# Grafo no dirigido simple con atributos de riesgo
class Graph:
    def __init__(self):
        self.adj = defaultdict(dict)  # adj[u][v] = {"risk": ...}
    def add_edge(self, u, v, risk: float):
        self.adj[u][v] = {"risk": float(risk)}
        self.adj[v][u] = {"risk": float(risk)}
    def neighbors(self, u):
        return self.adj[u].keys()
    def risk(self, u, v):
        return self.adj[u][v]["risk"]

# Reconstrucción de ruta
def reconstruct(parent, t):
    path = []
    n = t
    while n is not None:
        path.append(n)
        n = parent.get(n)
    return list(reversed(path))

# UCS multi-salida
# returns: path, total_risk, expanded, runtime_ms

def ucs_safest_path(G: Graph, start, exits: set):
    t0 = time.perf_counter()
    openq = []
    heappush(openq, (0.0, start))
    g = {start: 0.0}
    parent = {start: None}
    visited = set()
    expanded = 0
    while openq:
        cur_cost, u = heappop(openq)
        if u in visited:
            continue
        visited.add(u)
        expanded += 1
        if u in exits:
            dt = (time.perf_counter() - t0) * 1000
            return reconstruct(parent, u), g[u], expanded, round(dt, 3)
        for v in G.neighbors(u):
            c = g[u] + G.risk(u, v)
            if v not in g or c < g[v]:
                g[v] = c
                parent[v] = u
                heappush(openq, (g[v], v))
    return [], math.inf, expanded, round((time.perf_counter() - t0) * 1000, 3)

# Variante: múltiples orígenes (por ejemplo varios puntos donde hay personas)
# Equivalente a agregar un "súper-origen" conectado a cada origen con riesgo 0.

def ucs_multi_source(G: Graph, sources: set, exits: set):
    t0 = time.perf_counter()
    openq = []
    g = {}
    parent = {}
    visited = set()
    expanded = 0
    for s in sources:
        g[s] = 0.0
        parent[s] = None
        heappush(openq, (0.0, s))
    while openq:
        cur_cost, u = heappop(openq)
        if u in visited: continue
        visited.add(u)
        expanded += 1
        if u in exits:
            dt = (time.perf_counter() - t0) * 1000
            return reconstruct(parent, u), g[u], expanded, round(dt, 3)
        for v in G.neighbors(u):
            c = g[u] + G.risk(u, v)
            if v not in g or c < g[v]:
                g[v] = c
                parent[v] = u
                heappush(openq, (g[v], v))
    return [], math.inf, expanded, round((time.perf_counter() - t0) * 1000, 3)

# --- Ejemplo mínimo
if __name__ == "__main__":
    G = Graph()
    # Triángulo con una vía muy riesgosa y otra más segura
    G.add_edge('A', 'B', risk=0.9)  # alto riesgo
    G.add_edge('B', 'C', risk=0.8)
    G.add_edge('A', 'C', risk=0.2)  # bajo riesgo (ruta directa más segura)

    start = 'A'
    exits = {'C'}
    path, total_risk, expanded, ms = ucs_safest_path(G, start, exits)
    print({
        'path': path,
        'total_risk': round(total_risk, 3),
        'expanded': expanded,
        'runtime_ms': ms
    })

{'path': ['A', 'C'], 'total_risk': 0.2, 'expanded': 2, 'runtime_ms': 0.009}


In [16]:
from copy import deepcopy

# Accesores/políticas de riesgo (ejemplos)
def risk_expected(G, u, v):
    # usa el campo 'risk' base
    return G.adj[u][v]['risk']

def risk_robust_max(G, u, v):
    # si existen min/max, toma el peor caso; si no, cae al 'risk' base
    e = G.adj[u][v]
    return e.get('risk_max', e.get('risk', 0.0))

def risk_hurwitz(G, u, v, lam=0.7):
    e = G.adj[u][v]
    rmin = e.get('risk_min', e.get('risk', 0.0))
    rmax = e.get('risk_max', e.get('risk', e.get('risk', 0.0)))
    return lam * rmax + (1-lam) * rmin

# Wrapper UCS que acepta una función de riesgo
from heapq import heappush, heappop
import time, math

def ucs_with_policy(G, start, exits, risk_fn):
    t0 = time.perf_counter()
    openq = [(0.0, start)]
    g = {start: 0.0}
    parent = {start: None}
    visited = set()
    expanded = 0
    while openq:
        cur, u = heappop(openq)
        if u in visited:
            continue
        visited.add(u)
        expanded += 1
        if u in exits:
            # reconstrucción
            path = []
            x = u
            while x is not None:
                path.append(x)
                x = parent.get(x)
            path.reverse()
            return {
                'path': path,
                'total_risk': g[u],
                'expanded': expanded,
                'runtime_ms': round((time.perf_counter()-t0)*1000, 3)
            }
        for v in G.neighbors(u):
            c = g[u] + risk_fn(G, u, v)
            if v not in g or c < g[v]:
                g[v] = c
                parent[v] = u
                heappush(openq, (g[v], v))
    return {
        'path': [], 'total_risk': math.inf, 'expanded': expanded,
        'runtime_ms': round((time.perf_counter()-t0)*1000, 3)
    }

# Generador de tabla Markdown

def to_markdown(rows):
    cols = ['escenario','path','total_risk','expanded','runtime_ms']
    lines = ["| "+" | ".join(cols)+" |", "|"+"---|"*len(cols)]
    for r in rows:
        path_str = " → ".join(r['path']) if r['path'] else '—'
        lines.append(f"| {r['escenario']} | {path_str} | {round(r['total_risk'],3)} | {r['expanded']} | {r['runtime_ms']} |")
    return "\n".join(lines)

# Ejemplo de uso con tu grafo A-B-C
G2 = Graph()
G2.add_edge('A','B', 0.9)
G2.add_edge('B','C', 0.8)
G2.add_edge('A','C', 0.2)
# añade incertidumbre opcional
G2.adj['A']['C']['risk_min'] = 0.15
G2.adj['A']['C']['risk_max'] = 0.35

start, exits = 'A', {'C'}
rows = []
rows.append({ 'escenario':'Esperado', **ucs_with_policy(G2, start, exits, risk_expected) })
rows.append({ 'escenario':'Robusto(max)', **ucs_with_policy(G2, start, exits, risk_robust_max) })
rows.append({ 'escenario':'Hurwitz(0.7)', **ucs_with_policy(G2, start, exits, lambda G,u,v: risk_hurwitz(G,u,v,lam=0.7)) })

print(to_markdown(rows))

| escenario | path | total_risk | expanded | runtime_ms |
|---|---|---|---|---|
| Esperado | A → C | 0.2 | 2 | 0.053 |
| Robusto(max) | A → C | 0.35 | 2 | 0.006 |
| Hurwitz(0.7) | A → C | 0.29 | 2 | 0.006 |


## Respuestas a Preguntas Orientadoras